### **Extraccion Incremental de "Crypto Historical Market Depth API" de https://finage.co.uk/**
En este caso, ya que con **"Crypto Historical Market Depth API"** se busca obtener los trades entre las fechas especificadas de alguna criptomoneda con el fin de realizar una serie historica, y dado el volumen de los datos (sería bastante grande si las fechas distan lo suficiente entre si y si la API permitiría realizar un mayor volumen de consultas gratuitamente), se decidió utilizar el Metodo Extracción de Tipo Incremental.

In [1]:
#Importo todas las Librerias que utilizaré

import pandas as pd
from datetime import datetime, timedelta
from configparser import ConfigParser
from utils_API import *
from utils_state import *
from utils_parquet import *

In [2]:
#Leo el archivo JSON metadata_ingestion.json que contiene el ultimo valor incremental extraído 
#de la API con read_state_from_json() de utils_state.py y lo guardo en la variable state
state = read_state_from_json('metadata/metadata_ingestion.json')

#Obtengo el último valor incremental del JSON devuelto por la API en la ultima consulta con get_last_incremental_value() 
#de utils_state.py y utilizo ese valor como fecha inicial con la que hare la query
incremental_start_date = get_last_incremental_value(state, 'finage_API')

#Convierto la fecha inicial de string a un objeto datetime
incremental_start_date = datetime.strptime(incremental_start_date, '%Y-%m-%d %H:%M:%S')

#Asigno ese mismo obejeto a la fecha final y lo aumento en 1 hora con 'datetime.timedelta()' de datetime para 
#inicializar la fecha final con la que haré la query como 1 hora mas que la fecha inicial (asi lo pide la API)
incremental_end_date = incremental_start_date + timedelta(minutes=59)

#Les doy a la fecha final e inicial el formato indicado en la documentacion de la API
#Para ello obtengo los timestamp correspondientes en milisegundos y los convierto en string
incremental_start_date = str(int(incremental_start_date.timestamp() * 1000))
incremental_end_date = str(int(incremental_end_date.timestamp() * 1000))

#impresion de control
print(incremental_start_date, type(incremental_start_date))
print(incremental_end_date, type(incremental_end_date))

1577858542000 <class 'str'>
1577862082000 <class 'str'>


In [3]:
#Instancio ConfigParser() de configparser en la variable parser y con el leo el archivo de configuracion pipeline.conf
parser = ConfigParser()
parser.read('pipeline.conf')

#Extraigo la informacion de la seccion 'finage' de pipeline.conf
api_credentials = parser["finage"]

#Guardo la informacion extraida en sus variables correspondientes
api_key = api_credentials["api_key"]
limit = api_credentials["limit"] #El limite sera de 3 debido a que la API me permite solo 1.000 Requests por mes

In [4]:
#Establezco la url base y los parametros con la informacion extraida 
#de pipeline.conf para realizar la query segun la documentacion de la API
base_url = "https://api.finage.co.uk"
params = {'limit':limit,'apikey':api_key}

In [5]:
#Creo una lista con los simbolos de las criptomonedas, con montos 
#expresados en dolares, de las que solicitare los trades
symbols = ['btcusd', 'ethusd', 'adausd', 'dogeusd']

#Creo una lista donde guardare los dataframes con los trades de cada criptomoneda
df_crypto_trades = []

for symbol in symbols:
    #Modifico el endpoint para obtener informacion de los trades de cada criptomoneda en la lista symbols
    endpoint = f"history/crypto/depth/{symbol}/{incremental_start_date}/{incremental_end_date}"

    #hago la request con get_data() de utils_API.py
    json_data = get_data(base_url, endpoint, params=params)

    #Construyo un DataFrame con build_table() de utils_API.py 
    #usando la lista del JSON devuelto por la API
    df_current_crypto_trades = build_table(json_data)

    #Agrego la columna 'symbol' al DataFrame
    df_current_crypto_trades['symbol'] = symbol

    #Reordeno las columnas del DataFrame
    columns = ['symbol', 'p', 'q', 't']

    #Reindexo el DataFrame con las columnas en el nuevo orden para volverlo mas intuitivo
    df_current_crypto_trades = df_current_crypto_trades.reindex(columns=columns)

    #Impresion de control
    print(df_current_crypto_trades)

    #Agrego el dataframe de los ticks de la criptomoneda actual a la lista de dataframes df_crypto_ticks
    df_crypto_trades.append(df_current_crypto_trades)

   symbol              p           q              t
0  btcusd  7226.29000000  0.10698600  1577858543330
1  btcusd  7226.39000000  0.00152700  1577858543668
2  btcusd  7226.39000000  0.01347300  1577858545199
   symbol             p           q              t
0  ethusd  130.35000000  1.00000000  1577858543476
1  ethusd  130.35000000  1.86800000  1577858545632
2  ethusd  130.35000000  0.09010000  1577858546566
   symbol           p              q              t
0  adausd  0.03309000  1453.00000000  1577858599882
1  adausd  0.03310000   342.60000000  1577858599888
2  adausd  0.03309000   998.00000000  1577858600397
    symbol           p              q              t
0  dogeusd  0.00201430  5000.00000000  1577859669679
1  dogeusd  0.00201470  7408.00000000  1577859669679
2  dogeusd  0.00201470  6743.00000000  1577860398961


In [6]:
#Construyo un dataframe con todos los dataframes de cada criptomoneda 
#almacenados en df_cryptos con el metodo 'concat()' de pandas
df_incremental = pd.concat(df_crypto_trades, ignore_index=True)
df_incremental.head(12)

,symbol,p,q,t
0,btcusd,7226.29000000,0.10698600,1577858543330
1,btcusd,7226.39000000,0.00152700,1577858543668
2,btcusd,7226.39000000,0.01347300,1577858545199
3,ethusd,130.35000000,1.00000000,1577858543476
4,ethusd,130.35000000,1.86800000,1577858545632
5,ethusd,130.35000000,0.09010000,1577858546566
6,adausd,0.03309000,1453.00000000,1577858599882
7,adausd,0.03310000,342.60000000,1577858599888
8,adausd,0.03309000,998.00000000,1577858600397
9,dogeusd,0.00201430,5000.00000000,1577859669679


In [7]:
#Leo el archivo JSON metadata_ingestion.json que contiene el ultimo valor incremental extraído 
#de la API con read_state_from_json() de utils_state.py y lo guardo en la variable state
state = read_state_from_json('metadata/metadata_ingestion.json')

#Obtengo el mayor valor incremental del JSON devuelto por la API en la consulta actual con get_max_incremental_value() de utils_API.py.
new_value = get_max_incremental_value(df_incremental['t'])

#Actualizo el valor incremental del JSON en el estado de la replicación con update_incremental_value() de utils_state.py
update_incremental_value(state, 'metadata/metadata_ingestion.json', 'finage_API', new_value)

In [8]:
#Convierto la columna 't' de timestamp a tipo datetime (se necesita agregar unit='ms' porque los timestamp son de tipo numpy.int64)
df_incremental['t'] = pd.to_datetime(df_incremental['t'], unit='ms')

#Formateo los timestamps en la columna 't' como fechas con el formato 'YYYY-mm-dd HH-MM-SS'
df_incremental['t'] = df_incremental['t'].dt.strftime('%Y-%m-%d %H:%M:%S')

#Creo la columna hours con las horas de los trades para particionar 
#al guardar en el dataframe posteriormente en formato parquet
df_incremental['hours'] = pd.to_datetime(df_incremental['t']).dt.hour

#Imprimo el nuevo DataFrame
df_incremental.head(12)

,symbol,p,q,t,hours
0,btcusd,7226.29000000,0.10698600,2020-01-01 06:02:23,6
1,btcusd,7226.39000000,0.00152700,2020-01-01 06:02:23,6
2,btcusd,7226.39000000,0.01347300,2020-01-01 06:02:25,6
3,ethusd,130.35000000,1.00000000,2020-01-01 06:02:23,6
4,ethusd,130.35000000,1.86800000,2020-01-01 06:02:25,6
5,ethusd,130.35000000,0.09010000,2020-01-01 06:02:26,6
6,adausd,0.03309000,1453.00000000,2020-01-01 06:03:19,6
7,adausd,0.03310000,342.60000000,2020-01-01 06:03:19,6
8,adausd,0.03309000,998.00000000,2020-01-01 06:03:20,6
9,dogeusd,0.00201430,5000.00000000,2020-01-01 06:21:09,6


### **Almacenamiento de los DataFrames en un Data Lake en formato Parquet**

In [9]:
#Creo la ruta en donde se almacenaran los archivos parquet
bronze_dir = "datalake/bronze/finage_api"

In [10]:
#Guardo el DataFrame de la extraccion incremental como archivo parquet con save_to_parquet() de utils_parquet.py 
#en el directorio Crypto_Historical_Market_Depth de finage_api de la capa bronze del Data Lake particionado por hora
save_to_parquet(
    df_incremental,
    f"{bronze_dir}/Crypto_Historical_Market_Depth",
    "hours"
    )